In [1]:
import numpy as np
#for now, only euclidean and affine, no homogra

#from Ilya
import skimage.filters
import math
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.filters import sobel
from skimage import transform
from skimage import data, io, filters
from skimage import img_as_ubyte



In [2]:
def transform_point(input_pt, tfm_matrix):
    #if size(input_pt, 1)==1
    if input_pt.shape[0] == 1: #we make input into a column vector
        input_pt = input_pt.H
        b_row = 1 
    else:
        b_row = 0

    input_pt = np.asarray([input_pt, 1]) #please match shapes please
    output_pt = tfm_matrix.dot(input_pt)
    output_pt = output_pt/output_pt[2]
    output_pt = output_pt[0:2, 0]
    if b_row == 1:
        output_pt = output.H
        
    return output_pt

In [3]:
def tfm2para(tfm_matrix, XData, YData, mode):
    """tfm2para will transpose tfm_matrix to its corresponding parameter.
     -------------------------input------------------------------------------
     tfm_matrix:       3-by-3 matrix.
     mode:             one of 'euclidean', 'euclidean_notranslation', 'affine', 'affine_notranslation', 
                       'homography', 'homography_notranslation'
     -------------------------output-----------------------------------------
     tau:              p-by-1 real vector.
    """
    if mode == 'euclidean':
        tau = np.arccos(tfm_matrix[0, 0])
        if tfm_matrix[1, 0] < 0:
            tau *= -1.
        tau = np.array(tau)
        
    elif mode =='affine':
        tau = np.zeros((4, 1))
        tau[0:2] = tfm_matrix[0,0:2].H
        tau[2:] = tfm_matrix[1,0:2].H
        
    else: 
        print 'wut'
        
    return tau   

In [4]:
def para2tfm(tau, XData, YData, mode):
    """para2tfm will turn tau to tfm_matrix according to mode.
       ----------------------------input---------------------------------------
       tau:      p-by-1 vector
       mode:     one of 'euclidean', 'euclidean_notranslation', 'affine', 'affine_notranslation', 'homography',
                'homography_notranslation'
       ----------------------------output--------------------------------------
       tfm_matrix:   3-by-3 transform matrix.
    """
    tfm_matrix = np.eye(3)
    if mode == 'euclidean':
        tfm_matrix[0:2,0:2] = [[np.cos(tau[0]), -np.sin(tau[0])],
                               [np.sin(tau[0]), np.cos(tau[0])]]
    
    elif mode == 'affine':
        tfm_matrix[0,0:2] = tau[0:2].H
        tfm_matrix[1,0:2] = tau[2:].H
    
    else: 
        print 'no param'
    
    return tfm_matrix

In [5]:
def constraints(tau, XData, YData, mode):
    """constraints() will get the linearize constraints of tau according to mode.
    -----------------------------input--------------------------------------
    tau:          p-by-1 real vector.
    mode:         one of 'euclidean', 'euclidean_notranslation', 'affine', 'affine_notranslation', 'homography',
    'homography_notranslation'.
    ----------------------------output--------------------------------------
    linearized constraints on tau.

    """
    
    if mode == 'euclidean':
        S = np.zeros((2,1))
        
    return S
    #if mode == 'affine':
        


for reference
![title](screen.png)

#### S is a shrinkage operator
![title](screen_s.png)

In [6]:
def parse_TILT_arguments(**kwargs):
    my_args = kwargs
    
    if 'initial_tfm_matrix' not in my_args:
        my_args['initial_tfm_matrix'] = np.eye(3)
        
    if 'outer_tol' not in my_args:
        my_args['outer_tol'] = 1e-4
        
    if 'outer_max_iter' not in my_args:
        my_args['outer_max_iter'] = 50
        
    if 'outer_display_period' not in my_args:
        my_args['outer_display_period'] = 1
        
    if 'inner_tol' not in my_args:
        my_args['inner_tol'] = 1e-4
    
    if 'inner_c' not in my_args:
        my_args['inner_c'] = 1
        
    if 'inner_mu' not in my_args:
        my_args['inner_mu'] = []  #questionable
        
    if 'inner_display_period' not in my_args:
        my_args['inner_display_period'] = 100
        
    if 'inner_max_iter' not in my_args:
        my_args['inner_max_iter'] = np.inf
        
    if 'blur' not in my_args:
        my_args['blur'] = 1
        
    if 'branch' not in my_args:
        my_args['branch'] = 1
        
    if 'pyramid' not in my_args:
        my_args['pyramid'] = 1
        
    if 'focus_threshold' not in my_args:  #when doing pyramid the smallest focus_edge we can tolerate.
        my_args['focus_threshold'] = 50 
    
    if 'outer_tol_step' not in my_args:  #as resolution goes high how relaxed should the outer_tol be
        my_args['outer_tol_step'] = 10
        
    if 'blur_kernel_size_k' not in my_args:  # neighbourhood scalar for the size of the blur kernel.
        my_args['blur_kernel_size_k'] = 22
        
    if 'blur_kernel_sigma_k' not in my_args: # standard derivation scalar for blur kernel.
        my_args['blur_kernel_sigma_k'] = 2
        
    if 'pyramid_max_level' not in my_args: # number of pyramid levels we want to act on.
        my_args['pyramid_max_level'] = 2
        
    if 'branch_max_iter' not in my_args: # in each branch, how much iteration we take.
        my_args['branch_max_iter'] = 10
        
    if 'branch_max_accuracy' not in my_args: # higher means smaller step-width.
        my_args['branch_max_accuracy'] = 5
        
    if 'display_result' not in my_args:
        my_args['display_result'] = 1
        
    if 'focus_size' not in my_args:
        my_args['focus_size'] = []
        
    if 'save_path' not in my_args:
        my_args['save_path'] = []
        
    #MORE STUFF FOR BRANCH AND BOUND SKIPPED HERE! ADD LATER
    #Stuff skipped here because I do it outside
    
    return my_args

        
    

In [7]:
"""
% Version info: v 1.04
% -------------------------------------------------------------------------
% Update: 1. Improve the performance of the code on large images.
%         2. Simplify the modes of the code.
% -------------------------------------------------------------------------
% 
% 
% TILT will align the drawn-part of the image to its frontal.
% TILT is built on the kernel_component tilt_kernel.
% TILT(input_image, mode, based_points, focus_size) is the simplest form of
% the parameter, and these four parameters must be specified. There are
% also many other optional parameters.
%
% ----------------------necessary input------------------------------------
% input_image:  height-by-width real matrix or height*width*3 real matrix but
%               we will only preserve the first channel for the second
%               case.
% mode:         one of 'euclidean', 'affine', 'homography'.
% 
% [note]: One of the following sets of parameters must be specified.
% Set 1: initial_points:  
%               2-by-1 real vector in x-y co-ordinates, top-left point of
%               the focus. In this situation our algorithm will
%               automatically decide the UData, VData, XData and YData as
%               the output.
% Set 2: UData, VData, XData and YData.
%               All 1-by-2 real vector. These specifying the coordinate of
%               the input image and the transformed image, and we'll work
%               in this coordinate system by default.
% 
% ----------------------optional parameters--------------------------------
% INITIAL_MATRIX:   3-by-3 initialization matrix for tfm_matrix. If this is
%                   not set, tfm_matrix will be initialized to identity.
% BRANCH:           0 or 1, 1 for turnning on branch-and-bound, but this is
%                   only allowed in the AFFINE case.
% BLUR:             0 or 1, 1 for turnning on BLUR.
% PYRAMID:          0 or 1, 1 for turnning on pyramid.
% NO_TRANSLATION:   0 or 1, 1 for no translation.
% 
% INNER_TOL:        positive real value, inner_loop threshold.
% INNER_C:          positive real value, inner_loop lambda=C/sqrt(m).
% INNER_MU:         positive real value, inner_loop for ALM mu.
% INNER_MAXITER:    positive integer, maximum iteration for inner_loop.
% INNER_DISPLAY:    positive integer, whether we display the results.
% OUTER_TOL:        positive real value, outer_loop threshold.
% OUTER_MAXITER:    positive integer, maximum iteration for ouer_loop.
% OUTER_DISPLAY:    positive integer, whether we display the results for
%                   the outer loop.
% FOCUS_THRE:       positive integer, smallest edge length threshold in
%                   pyramid
% OUTER_TOL_STEP:   positive real value, We relax threshold for
%                   outer-loop each time we move downstairs in pyramid
%                   by tol=tol*OUTER_TOL_STEP
% BLUR_SIGMA:       positive real value, standard derivation of the blur
%                   kernel.
% BLUR_NEIGHBOR:    positive integer, size of effective blur neighbourhood.
% BRANCH_MAXITER:   positive integer, we need have extremely high accuracy
%                   in branch-and=bound. So we separately set
%                   branch_maxiter for branch_and_bound.
% BRANCH_ACCURACY:  positive integer, we split the whole parameter region
%                   to search into 2*branch_accuracy+1 sub-region.
% BRANCH_MAX_ROTATION:
%                   positive real, by default pi/8, specifying how large to
%                   do branch-and-bound for rotation.
% BRANCH_MAX_SKEW:
%                   positive, real, by default 
% PYRAMID_MAXLEVEL: positive integer, we only run TILT on the highest
%                   PYRAMID_MAXLEVEL levels in the pyramid.
% DISPLAY_INTER:    0 or 1, whether we display results for every call to
%                   tilt_kernel.m
% FOCUS_SIZE:       1*2 row vector, forcing focus_size to be something.
% SAVE_PATH:        string, full path of where to save the results.
% 
% -------------------------output------------------------------------------
% Dotau:        real matrix with same size as focus_size, aligned images.
% A:            low-rank part of Dotau;
% E:            sparse-error part of Dotau;
% f:            value of objective-function;
% tfm_matrix:   resulted transform matrix.
% focus_size:   1*2 positive integer vector, size of the focus window in
%               r-c coordinate.
% error_sign:   0 or 1, 1 for trival solutions.
% UData, VData: 1*2 real vector, position of the input image.
% XData, YData: 1*2 real vector, position of the transformed image.
"""
###SOMEWHERE! I NEED to CHECK FOR MAX AND AMAX!!!

"\n% Version info: v 1.04\n% -------------------------------------------------------------------------\n% Update: 1. Improve the performance of the code on large images.\n%         2. Simplify the modes of the code.\n% -------------------------------------------------------------------------\n% \n% \n% TILT will align the drawn-part of the image to its frontal.\n% TILT is built on the kernel_component tilt_kernel.\n% TILT(input_image, mode, based_points, focus_size) is the simplest form of\n% the parameter, and these four parameters must be specified. There are\n% also many other optional parameters.\n%\n% ----------------------necessary input------------------------------------\n% input_image:  height-by-width real matrix or height*width*3 real matrix but\n%               we will only preserve the first channel for the second\n%               case.\n% mode:         one of 'euclidean', 'affine', 'homography'.\n% \n% [note]: One of the following sets of parameters must be specified.\n% 

In [8]:
def TILT(input_image, mode, init_points, **kwargs):
    
    #Note - I am really concerned that there will be mix up between (N,) and (N,1) type arrays, python doesn't like it
    args = parse_TILT_arguments(**kwargs)
    
    #doing some initialization
    #this is from inside parse, just because
    #I assume init_points is (2,2) array
    args['mode'] = mode
    args['input_image'] = input_image
    args['initial_points'] = np.floor(init_points)
    args['focus_size'] = np.asarray([init_points[1,1]-args['initial_points'][1,0], args['initial_points'][0,1]-args['initial_points'][0,0]])
    #above - because of different array conventions, 1 might be needed or not, I am confused honestly
    args['center'] = np.floor(np.mean(init_points, axis=1)) #I seriously hope conventions are the same
    args['focus_center'] = np.floor(np.asarray([args['focus_size'][1],  args['focus_size'][0]])/2.) #onwes were removed
    focus_center = args['focus_center']
    XData = np.asarray([1-focus_center[0], args['focus_size'][1]-focus_center[0]])
    YData = np.asarray([1-focus_center[1], args['focus_size'][0]-focus_center[1]])
    #now, afterwards there is nasty stuff about compute homography
    #hopefully, I can avoid it and initialize tfm to identity
    

    original_args = args
    
    
    #creating boundaries of the image
    expand_rate = 0.8 #WTF is this shit
    initial_points = init_points
    left_bound = np.ceil(max(initial_points[0,0] - expand_rate * (initial_points[0,1] - initial_points[0,0]), 0))
    right_bound = np.floor(min(initial_points[0,1] + expand_rate * (initial_points[0,1] - initial_points[0,0]), input_image.shape[1]));
    top_bound = np.ceil(max(initial_points[1,0] - expand_rate * (initial_points[1,1] - initial_points[1,0]), 0))
    bottom_bound = np.floor(min(initial_points[1,1] + expand_rate*(initial_points[1,1] - initial_points[1,0]), input_image.shape[0]));
    new_image = np.zeros((bottom_bound - top_bound + 1, right_bound - left_bound + 1, input_image.shape[2])) #our image has color
    
    #for c in range(input_image.shape[2]):
    #    new_image[:,:,c]=input_image[top_bound:bottom_bound+1, left_bound:right_bound+1, c] #maybe miss one pixel? but whatever
    
    new_image = input_image #TEMP
    
    #note - here I didn't copy this: args.input_image=uint8(new_image), 
    #because that would be handled at input by scikit
    #sorry, to tired to make it not bydlocode
    #args['input_image'] = img_as_ubyte(new_image) #hope this is right
    args['input_image'] = new_image
    args['center'] = args['center'] + np.asarray([1-left_bound, 1-top_bound])  #verrry not sure about that one
    
    
    pre_scale_matrix=np.eye(3) 
    #here should be the part "down-sample the image if the focus is too large"
    #but guess what
    #it won't be too large
    #at least for initial testing
    min_length = original_args['focus_size']
    
    #adjust the parsed parameters
    initial_tfm_matrix = args['initial_tfm_matrix']
    initial_tfm_matrix = np.linalg.inv(pre_scale_matrix).dot(initial_tfm_matrix).dot(pre_scale_matrix)
    args['initial_tfm_matrix'] = initial_tfm_matrix
    args['focus_size'] = np.around(args['focus_size']/pre_scale_matrix[0,0])
    args['center'] = args['center']/pre_scale_matrix[0,0]
    #parent_path=args['save_path']
    parent_path='./'
    #NOW ILYA PART
    #if args['blur']==1:
    #    img_size=np.shape(args['input_image'])
    #    img_size=img_size[:2]
    #    input_image = gaussian(args['input_image'],sigma = math.ceil(args['blur_kernel_sigma_k']*max(img_size)/50))
    #       #here might be some problems with filter 
    #else:
        #input_image=args['input_image']
    input_image=args['input_image']
    args['figure_no'] = 101
    args['save_path'] = os.path.join(parent_path, 'some_name')
    print args['center']
    print args['focus_size']
    Dotau, A, E, tfm_matrix, UData, VData, XData, YData, A_scale = tilt_kernel(input_image, args['mode'], args['center'], args['focus_size'], args['initial_tfm_matrix'], args)
    
    args = original_args
    tfm_matrix = np.dot(pre_scale_matrix,np.dot(tfm_matrix,np.linalg.inv(pre_scale_matrix)))
    
    focus_size=args['focus_size']
    image_size=np.shape(args['input_image'])
    image_size=image_size[:2]
    image_center=args['center']
    focus_center=np.zeros((2, 1))
    focus_center[0]=np.floor((1+args['focus_size'][1])/2)
    focus_center[1]=np.floor((1+args['focus_size'][0])/2)
    #UData=[1-image_center[0], image_size[1]-image_center[0]]  #This makes lists, not arrays
    #VData=[1-image_center[1], image_size[0]-image_center[1]]  #below is my correction
    #XData=[1-focus_center[0], args.focus_size[1]-focus_center[0]] #which I hope is right
    #YData=[1-focus_center[1], args.focus_size[0]-focus_center[1]]
    
    UData=np.asarray([1-image_center[0], image_size[1]-image_center[0]]) 
    VData=np.asarray([1-image_center[1], image_size[0]-image_center[1]])
    XData=np.asarray([1-focus_center[0], args['focus_size'][1]-focus_center[0]])
    YData=np.asarray([1-focus_center[1], args['focus_size'][0]-focus_center[1]])
    
    #display the result    #I NEED TO MAKE THE SAME CORRECTION BELOW - P.
    #if args['display_result'] == 1:
       # plt.figure(99)
        #plt.imshow(args[input_image], [], 'DisplayRange', [0, max(max(max(args.input_image)))])
        # I don't what are parameters are here, may be we should omit them
        #X1=[args['initial_points'][0,0], args['initial_points'][0,0], args['initial_points'][0,1], args['initial_points'][0,1], args['initial_points'][0,0]]
        #Y1=[args['initial_points'][1,0], args['initial_points'][1,1], args['initial_points'][1,1], args['initial_points'][1,0], args['initial_points'][1,0]]
        #plt.hold(True)
        #plt.plot(X1, Y1, 'r-')
        #pt_top_left = transform_point([XData[0], YData[0]], tfm_matrix)+image_center
        #pt_bottom_left = transform_point([XData[0], YData[1]], tfm_matrix)+image_center
        #pt_bottom_right = transform_point([XData[1], YData[1]], tfm_matrix)+image_center
        #pt_top_right = transform_point([XData[1], YData(1)], tfm_matrix)+image_center
        #X2=[pt_top_left[0], pt_bottom_left[0], pt_bottom_right[0], pt_top_right[0], pt_top_left[0]]
        #Y2=[pt_top_left[1], pt_bottom_left[1], pt_bottom_right[1], pt_top_right[1], pt_top_left[1]]
        #plt.plot(X2, Y2, 'g-')
    
    return  Dotau, A, E

In [9]:
def tilt_kernel(input_image, mode, center, focus_size, initial_tfm_matrix, para):
    '''
        % tilt_kernel aligns a sub-image of input_image specified by base_points
        % and focus_size to its frontal, with low-column rank subject to some
        % linear constraints.
        % -------------------------input------------------------------------------
        % input_image:  height-by-width real matrix or height*width*3 real matrix but
        %               we will only preserve the first channel for the second
        %               case.
        % mode:         one of 'euclidean', 'euclidean_notranslation', 'affine', 'affine_notranslation', 'homography',
        %               'homography_notranslation'.
        % center:       2-by-1 real vector in row-column co-ordinates, origin of
        %               the image.
        % focus_size:   1-by-2 real vector in row-column co-ordinates, size of the
        %               focus.
        % initial_tfm_matrix:   3-by-3 matrix, optional, the initial transform
        %                       matrix. If not specified set it to be eye(3);
        % para:         parameters for both inner-loop and outer-loop, must be
        %               specified!
        %
        % -------------------------output-----------------------------------------
        % Dotau:        real matrix with same size as focus_size, aligned images.
        % A:            low-rank part of Dotau;
        % E:            sparse-error part of Dotau;
        % f:            value of objective-function;
        % tfm_matrix:   resulted transform matrix.
        % error_sign:   0 or 1, 1 for trival solutions.
    '''

    outer_tol = 5e-5
    outer_max_iter = 50
    outer_display_perioud = 1    
    input_image =input_image[:,:,0]
    image_size = input_image.shape
    image_center = center.astype(int)
    focus_center = np.zeros((2,1))
    focus_center[0] = int((1+focus_size[1])/2)
    focus_center[1] =int((1+focus_size[0])/2)
    A_scale = 0.8
    
    #print "II: {0}".format(np.sum(input_image))
    
    #print 'focus center: {0}'.format(focus_center)
    
    UData = [1-image_center[0], image_size[1]-image_center[0]]
    VData = [1-image_center[1], image_size[0]-image_center[1]]
    XData = [1-focus_center[0], focus_size[1]-focus_center[0]]
    YData = [1-focus_center[1], focus_size[0]-focus_center[1]]
    
    input_du = sobel(input_image, 0)
    input_dv = sobel(input_image, 1)
    
    Dotau_series = []
    
    tfm_matrix=initial_tfm_matrix
    #tfm=fliptform(maketform('projective', tfm_matrix'));
    ## create inverse transform 
    tfm_matrix = initial_tfm_matrix.T
    trfrm = transform.SimilarityTransform(tfm_matrix)
    Dotau = transform.warp(input_image, trfrm)
    
    #print "dotau: {0}".format(np.sum(Dotau))
    
    Dotau_series.append(Dotau)
    initial_image = Dotau
    
    du = transform.warp(input_du, trfrm)
    dv = transform.warp(input_dv, trfrm)
    
    du= du / np.linalg.norm(Dotau, 'fro') - (sum(sum(Dotau*du))) / (np.linalg.norm(Dotau, 'fro')**3) * Dotau
    dv= dv / np.linalg.norm(Dotau, 'fro') - (sum(sum(Dotau*dv))) / (np.linalg.norm(Dotau, 'fro')**3) * Dotau
    
    A_scale = np.linalg.norm(Dotau, 'fro')
    Dotau = Dotau.astype(float) / np.linalg.norm(Dotau, 'fro')
    #print "dotau_upd: {0}".format(np.sum(Dotau))
    tau = tfm2para(tfm_matrix, XData, YData, mode)
    J = jacobi(du, dv, XData, YData, tau, mode)
    S = constraints(tau, XData, YData, mode)

    outer_round=0
    pre_f=0

    while 1:
        outer_round=outer_round+1
        A, E, delta_tau = inner_IALM_constraints(Dotau, J, S)
        #print('asas')
        #if error_sign == -1:
        #    return
        #end
       
        #if mod(outer_round, outer_display_period) == 0:
        #    disp(['outer_round ',num2str(outer_round),  ', rank(A)=', num2str(rank(A)), ', ||E||_1=', num2str(sum(sum(abs(E))))]);
        #print 'tau dtau', tau, delta_tau
        tau=tau + delta_tau
        
        
        tfm_matrix=para2tfm(tau, XData, YData, mode).T
        #tfm=fliptform(maketform('projective', tfm_matrix'))
        trfrm = transform.SimilarityTransform(tfm_matrix)
        Dotau = transform.warp(input_image, trfrm)
        
        #print "tfm mat\n", tfm_matrix
        #print "dotau_upd: {0}".format(np.sum(Dotau))
        
        Dotau_series.append(Dotau)
        # judge convergence
        if (outer_round >= outer_max_iter): #or (abs(f-pre_f) < outer_tol):UPDATE STOPCOND
            break
        
        #pre_f=f;
        
        du = transform.warp(input_du, trfrm)
        dv = transform.warp(input_dv, trfrm)

        du= du / np.linalg.norm(Dotau, 'fro') - (sum(sum(Dotau*du))) / (np.linalg.norm(Dotau, 'fro')**3) * Dotau
        dv= dv / np.linalg.norm(Dotau, 'fro') - (sum(sum(Dotau*dv))) / (np.linalg.norm(Dotau, 'fro')**3) * Dotau
        A_scale = np.linalg.norm(Dotau, 'fro')
        Dotau = Dotau / np.linalg.norm(Dotau, 'fro')

        tau = tfm2para(tfm_matrix, XData, YData, mode)
        J = jacobi(du, dv, XData, YData, tau, mode)
        S = constraints(tau, XData, YData, mode)

    
    
    #Dotau=imtransform(input_image, tfm, 'bilinear', 'UData', UData, 'VData', 
    #VData, 'XData', XData, 'YData', YData, 'size', focus_size);
    #Dotau_series{1}=Dotau;
    #initial_image=Dotau;
    return Dotau, A, E, tfm_matrix, UData, VData, XData, YData, A_scale                  

In [10]:
def jacobi(du, dv, XData, YData, tau, mode):
    """ jacobi() will calculate the jacobi matrix.
    ------------------------------input--------------------------------------
    du:           m-by-n matrix, derivative along x-axis;
    dv:           m-by-n matrtix, derivative along y-axis;
    UData:        1-by-2 vector, X-range of the image.
    VData:        1-by-2 vector, Y-range of the image.
    tfm_matrix:   3-by-3 matrix, transformation matrix.
    mode:         one of 'affine', 'affine_notranslation', 'homography',
    'homography_notranslation'
    ------------------------------output-------------------------------------
    J:            m-by-n-by-p tensor, jacobi matrix.

    """
    [m, n] = du.shape
    [X0, Y0] = np.meshgrid(range(XData[0],XData[1]+1), range(YData[0],YData[1]+1))

    if mode == 'euclidean':
        tau = [tau]
        J = np.zeros((m, n, 1))
        J[:,:,0] = du * (- X0 * np.sin(tau[0]) - Y0 * np.cos(tau[0])) + dv * (X0 * np.cos(tau[0]) - Y0 * np.sin(tau[0]))
    #if mode == 'affine':  
    
    return J
        

In [11]:
def inner_IALM_constraints(Dotau, J, S_J, tol=1e-7, c=1., mu=-5., max_iter=np.inf):
    #well shit then I guess I need to check stuff mu=1.25/np.linalg.norm(Dotau)
    """
    inner_IALM_constraints will solve the programming:
    min ||A||_*+lambda*||E||_1   s.t.     Dotau+J*delta_tau=A+E and
                                          S*delta_tau=0
    via the Inexact ALM method.
    ---------------------------------input----------------------------------
    Dotau:            m-by-n image matrix.
    J:                m-by-n-by-p tensor.
    S_J:                c-by-p matrix.
    inner_para:       parameters.
    --------------------------------output----------------------------------
    A:                m-by-n matrix, low-rank part.
    E:                m-by-n matrix, error part.
    delta_tau:        step of tau.
    f:                objective funtion value.
    """
    
    #prep data
    #sorry for a bit of bydlokod
    if mu == -5.:
        mu=1.25/np.linalg.norm(Dotau)
        
    [m, n] = Dotau.shape
    E = np.zeros((m, n))
    A = np.zeros((m, n))
    p = J.shape[2]
    delta_tau = np.zeros((p,1))
    
    J_vec = np.reshape(J, (m*n,1))#check whether reshape is the same
    Jo = J_vec
    #print 'J-vec before', J_vec.shape, S_J.shape,

    #J_vec = np.asarray([J_vec, S_J]) #plz check here as well, prob use smth else IS THIS TRUE I AM NOT SURE
    J_vec = np.vstack((J_vec, S_J))
    #print J_vec.shape

    #print 'Together', 
    pinv_J_vec = np.linalg.pinv(J_vec)
    inner_round = 0
    rho = 1.25
    lmbda = c / np.sqrt(m)
    
    Y_1 = Dotau
    #print "Y_1:\n:", Y_1
    norm_two = np.linalg.norm(Y_1, 2)
    norm_inf = np.linalg.norm(Y_1, np.inf)/lmbda #there was some mystery, what does norm_inf=norm(Y_1(:), inf)/lambda mean
    dual_norm = max(norm_two, norm_inf)
    Y_1 = Y_1 / dual_norm;
    Y_2 = np.zeros((S_J.shape[0], 1)) #this needs to be checked
    d_norm = np.linalg.norm(Dotau, 'fro')
    error_sign = 0
    #first_f = np.sum(np.linalg.svd(Dotau)) #this is so weird and I don't understand its purpose
    #stuff stuff stuff stuff stuff
    
    
    #let's now see what's in the main loop
    #ill just initialize stop_criterion to 10tol or something
    stop_criterion = 100*tol
    max_iter = 50
    while (stop_criterion > tol) and (inner_round < max_iter):
        
        inner_round += 1
        temp_0 = Dotau + np.reshape(np.dot(Jo, delta_tau), (m,n)) + Y_1 / mu
        temp_1 = temp_0 - E
        U, S, V = np.linalg.svd(temp_1, full_matrices=False)
        shrinkage_S =(S > 1/mu).astype(int) * (S - 1/mu) #because S is non-negative
        A = U.dot(shrinkage_S).dot(V.T)
        temp_2 = temp_0 - A
        E = (temp_2 > 1/mu).astype(int) * (temp_2 - 1/mu) + (temp_2 < -1/mu).astype(int) * (temp_2 + 1/mu)
        #here is some MYSTERIOUS f, I do not understand its purpose
        temp_3 = A + E - Dotau - Y_1 / mu
        temp_3 = np.reshape(temp_3, (m*n,1))
        #temp_3 = np.asarray([temp_3, -Y_2/mu]) #CHECK CHECK CHECK CHECK
        temp_3 = np.vstack((temp_3, -Y_2/mu))
        
        #print 'ialm',pinv_J_vec,temp_3
        delta_tau = pinv_J_vec.dot(temp_3)
        derivative_Y_1 = Dotau - A - E + np.reshape(Jo.dot(delta_tau), (m, n))
        derivative_Y_2 = S_J.dot(delta_tau)
        Y_1 = Y_1 + derivative_Y_1 * mu  #mu is scalar
        Y_2 = Y_2 + derivative_Y_2 * mu
        
        stop_criterion=np.sqrt(np.linalg.norm(derivative_Y_1, 'fro')**2 + np.linalg.norm(derivative_Y_2, 2)**2)/d_norm
        mu=mu*rho
        
        #there is some mysterious error-catching, I didn't understand that part
        
    return A, E, delta_tau

In [12]:
check = io.imread('tiny_check.jpg')
init_points = np.asarray([[0, 50], [0, 50]])
#print check.shape
checkie = np.zeros((50,50,3))
checkie[:,:,0] = check
checkie[:,:,1] = check
checkie[:,:,2] = check

#print checkie.shape

#print check[14,15]

#print img_as_ubyte(check)[14, 15]




Dotau, A, E = TILT(checkie, 'euclidean', init_points)

/home/polina/anaconda2/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[ 26.  26.]
[ 50.  50.]


In [13]:
plt.imshow(Dotau, cmap='gray')
plt.show()

In [14]:
Dotau

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])